# Batched Portfolio Optimization with Dual Variables

This notebook demonstrates two key CVXPYlayers features:

1. **Batched solving** — solve many problem instances simultaneously by passing batched parameters
2. **Dual variable extraction** — retrieve shadow prices from constraints

**Problem**: Markowitz mean-variance portfolio optimization with a parametric risk budget.

$$\max_{w} \; \mu^T w \quad \text{s.t.} \; w^T \Sigma w \leq \sigma_{\max}^2, \; \mathbf{1}^T w = 1, \; w \geq 0$$

**Key API**:
- Batched parameters: pass tensors with shape `(batch_size, ...)`
- Broadcasting: mix batched and unbatched parameters
- `constraint.dual_variables[0]` — include dual variables in the output

In [ ]:
import cvxpy as cp
import matplotlib.pyplot as plt
import numpy as np
import torch

from cvxpylayers.torch import CvxpyLayer

## Define the Portfolio Problem

We include the risk constraint's dual variable in the output. The dual value represents the *shadow price* of risk — how much additional expected return we'd get per unit relaxation of the risk constraint.

In [ ]:
n_assets = 5

w = cp.Variable(n_assets, name="weights")
mu = cp.Parameter(n_assets, name="expected_returns")
sigma_max = cp.Parameter(pos=True, name="risk_budget")

# Generate a fixed covariance matrix
np.random.seed(42)
F = np.random.randn(n_assets, n_assets) * 0.1
Sigma = F.T @ F + 0.05 * np.eye(n_assets)
Sigma_sqrt = np.linalg.cholesky(Sigma)

# Risk constraint: ||Sigma^{1/2} w||_2 <= sigma_max (SOC form)
risk_constraint = cp.norm(Sigma_sqrt.T @ w, 2) <= sigma_max

constraints = [
    risk_constraint,
    cp.sum(w) == 1,
    w >= 0,
]

objective = cp.Maximize(mu @ w)
problem = cp.Problem(objective, constraints)
assert problem.is_dpp()

# Include the dual variable for the risk constraint
risk_dual = risk_constraint.dual_variables[0]
layer = CvxpyLayer(
    problem,
    parameters=[mu, sigma_max],
    variables=[w, risk_dual],
)
print(f"Layer outputs: optimal weights (dim {n_assets}) + risk dual (scalar)")

## Unbatched Solve

First, solve a single instance to verify correctness.

In [ ]:
mu_tch = torch.tensor([0.12, 0.10, 0.07, 0.03, 0.02], dtype=torch.float64)
sigma_max_tch = torch.tensor(0.15, dtype=torch.float64)

w_opt, dual_opt = layer(mu_tch, sigma_max_tch)

print("Optimal weights:", w_opt.detach().numpy().round(4))
print(f"Portfolio return: {(mu_tch @ w_opt).item():.4f}")
print(f"Portfolio risk:   {torch.norm(torch.tensor(Sigma_sqrt.T) @ w_opt).item():.4f}")
print(f"Risk budget:      {sigma_max_tch.item():.4f}")
print(f"Risk dual value:  {dual_opt.detach().numpy().round(4)}")

## Batched Solve: Multiple Return Forecasts

Pass a batch of expected return vectors. The first dimension is the batch dimension.

In [ ]:
batch_size = 8
torch.manual_seed(0)

# Batch of return forecasts: shape (batch_size, n_assets)
# Use abs to ensure positive expected returns (realistic for financial assets)
mu_batch = torch.abs(torch.randn(batch_size, n_assets, dtype=torch.float64) * 0.03 + 0.06)

# Unbatched risk budget — automatically broadcast across the batch
sigma_max_tch = torch.tensor(0.20, dtype=torch.float64)

w_batch, dual_batch = layer(mu_batch, sigma_max_tch)

print(f"Input  mu shape:       {mu_batch.shape}")
print(f"Output weights shape:  {w_batch.shape}")
print(f"Output dual shape:     {dual_batch.shape}")
print()
print("Weights sum to 1 for each batch element:")
print(w_batch.sum(dim=1).detach().numpy().round(6))

## Broadcasting: Sweep Risk Budget

Use a batched `sigma_max` with an unbatched `mu` to trace the efficient frontier. CVXPYlayers broadcasts the unbatched parameter across the batch automatically.

In [ ]:
mu_fixed = torch.tensor([0.12, 0.10, 0.07, 0.03, 0.02], dtype=torch.float64)

# Sweep risk budgets above the minimum feasible risk (~0.148)
n_points = 40
sigma_values = torch.linspace(0.15, 0.45, n_points, dtype=torch.float64)

returns = []
risks = []
duals = []

for sig in sigma_values:
    w_sol, d_sol = layer(mu_fixed, sig)
    ret = (mu_fixed @ w_sol).item()
    risk = torch.norm(torch.tensor(Sigma_sqrt.T, dtype=torch.float64) @ w_sol).item()
    returns.append(ret)
    risks.append(risk)
    duals.append(d_sol.detach().numpy().flatten()[0])

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(11, 4))

ax1.plot(risks, returns, 'b-o', markersize=3, linewidth=2)
ax1.set_xlabel('Portfolio Risk (std dev)')
ax1.set_ylabel('Expected Return')
ax1.set_title('Efficient Frontier')
ax1.grid(True, alpha=0.3)

ax2.plot(sigma_values.numpy(), duals, 'r-o', markersize=3, linewidth=2)
ax2.set_xlabel('Risk Budget (sigma_max)')
ax2.set_ylabel('Dual Value (Shadow Price of Risk)')
ax2.set_title('Risk Constraint Dual')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
print("The dual value decreases as the risk budget relaxes (constraint becomes slack).")

## Verify: Dual Values Match Gradients

The dual value of the risk constraint should equal the gradient of the optimal return with respect to the risk budget (by the envelope theorem).

In [ ]:
# Compute gradient d(return)/d(sigma_max) and compare to dual
sigma_test = torch.tensor(0.15, dtype=torch.float64, requires_grad=True)
w_sol, dual_sol = layer(mu_fixed, sigma_test)
opt_return = mu_fixed @ w_sol
opt_return.backward()

print(f"Gradient d(return)/d(sigma_max): {sigma_test.grad.item():.6f}")
print(f"Dual value of risk constraint:   {dual_sol.detach().numpy().flatten()[0]:.6f}")
print(f"These should be close (envelope theorem).")